I have a prototype of they system for corrupting sentences, for different semantic meaning:

My process is:

1. Tokenize the sentence (Currently using the NLTK regex tokenizer, it seem sufficient)
2. Parts of Speech tag (Currently using the Stanford POS Tagger (via NLTK))
3. For each word except auxiliary verbs, and words that make up phrases (phrases are checked for by querying wordnet)
        1. Use WordNet to find antonyms of the same POS tag  (So "race" (Noun, as in a competition) has no antonyms, but "race" (Verb, to race the train) has "linger" as an antonym.
        2. Unstem: WordNet stemming/lemmaisation (of the antonym) removes Tense, Plurality, comparativeness, and superlativeness, so I make use of the POS tag of the original to work those out, then restore them using the Pattern library's tools for this (http://jmlr.csail.mit.edu/papers/volume13/desmedt12a/desmedt12a.pdf)
        3. Remove any generated words that are not real (Failures of the Unstemming process, eg saless), by checking against the SCOWL british-insane word list.
        4. I remove any suggested antonyms that are short phrases (eg Wordnet suggests that "take_away" is an antonym of "add", however as adding a work word change the structure of the sentence.)
        5. select one randomly if there are multiple.
4. I incrementally subsitute one addional antynym until I have run out of words with antonys, saving the sentence at each step. (What I'm currentl;y callign each sentence of different corruption level)
5. I repair the indefinite articles ('an' vs 'a')

I can check the final sentence by sending it through the POS tagger and seeing if I get the same tags.
This is not perfect. Its not bad though.
It got a lot better when I changed to using the Stanford POS tagger, as it was more able to tag and retag correctly and thus was most consistent.

The whole method is very heurustic.
It's not too bad but makes a lot of unnatural sounding sentences.
And will sometimes choose the wrong antonym.

For example stating with the sentence:
 - It's not too bad but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing acquittals.

In [1]:
from SemanticCorruption import *

In [2]:
from nltk.corpus import wordnet as wn
import pattern.en as en
import numpy as np

In [3]:
sents = ["It's not too bad but makes a lot of unnatural sounding sentences.",
'A Washington County man may have the countys first human case of West Nile virus, the health department said Friday',
"It was a dusty, dry, hot day, and the flys were buzzing.",
"The article is the most common determiner (DT) in English.",
"We may have a question",
"Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?",
"Both gerunds and infinitives can be used as the subject or the complement of a sentence.",
"Shares of Xoma fell 16 percent in early trade, while shares of Genentech, a much larger company with several products on the market, were up 2 percent.",
"Six months ago, the IMF and Argentina struck a bare-minimum $6.8-billion debt rollover deal that expires in August.",
"He plans to have dinner with troops at Kosovo's U.S. military headquarters, Camp Bondsteel.",
"After that, he plans to have dinner at Camp Bondsteel with U.S. troops stationed there.",
"He added that prosecutors will seek the death penalty.",
"Who is this man?",
"Who is that man?",
"This evil thief stole that car!",
"The motorist is angry, so the pedestrian is understandably scared",
"The hero was brave, he defeated the dragon.",
"The villian was cowardly, he was slain by the dragon.",
"War and Peace opens in the Russian city of St. Petersburg in 1805, as Napoleon's conquest of western Europe is just beginning to stir fears in Russia.",
"PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer.",
"You sing very well.",
"We look forward to your talk.",
"I knew French.",
         
        ]

In [10]:

def leveled_semantic_corrupt_noun_synonym_sentences(sent):
    words, tagged_words = tokenize_and_tag(sent)
    short_phrase_indexes = get_phrases_indexes(tagged_words,3)
    noun_corrupted_phases = leveled_semantic_corrupt_sentences_from_pretagged(words, tagged_words, get_verb_antos, short_phrase_indexes)
    return list(noun_corrupted_phases)


In [11]:

for ii,sent in enumerate(sents):
    print(ii)
    print(sent)
    print("\n".join(leveled_semantic_corrupt_noun_synonym_sentences(sent)))
    print("-"*50)

0
It's not too bad but makes a lot of unnatural sounding sentences.
It 's not too bad but makes a lot of unnatural devoicing sentences .
It 's not too bad but unmakes a lot of unnatural devoicing sentences .
--------------------------------------------------
1
A Washington County man may have the countys first human case of West Nile virus, the health department said Friday

--------------------------------------------------
2
It was a dusty, dry, hot day, and the flys were buzzing.

--------------------------------------------------
3
The article is the most common determiner (DT) in English.

--------------------------------------------------
4
We may have a question

--------------------------------------------------
5
Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?
Is staying an odd number of verbs , adverbs , adjectives and nouns to their antonyms expected to produce a semantically distant sen

In [64]:
list(get_all_antonyms("tell"))

[]

In [75]:
tags = set()
for sent in ["he has gone"]:# sents[-1:]:
    words,tagged_sent = tokenize_and_tag(sent)
    tags.update(zip(*tagged_sent)[1])
tags

{u'PRP', u'VBN', u'VBZ'}

In [80]:
sent = "Cadets were ticketted for drinking alcohol."
words,tagged_sent = tokenize_and_tag(sent)
tagged_sent

[(u'Cadets', u'NNS'),
 (u'were', u'VBD'),
 (u'ticketted', u'VBN'),
 (u'for', u'IN'),
 (u'drinking', u'NN'),
 (u'alcohol', u'NN'),
 (u'.', u'.')]

In [95]:
x.lemmas()

[Lemma('policeman.n.01.policeman'),
 Lemma('policeman.n.01.police_officer'),
 Lemma('policeman.n.01.officer')]

In [74]:
list(get_all_synonyms_of_most_common('drinking',wn.NOUN))

[u'drinking', u'imbibing', u'imbibition']

In [4]:
import sys
sys.path.append("../../../Resources/tools/lib/python/pywsd/")
import pywsd
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim

In [5]:
def make_pywsd_sent(tagged_words):
    return[(word,word,pennPOS2WordnetPOS(tag)) for (word,tag) in tagged_words]

In [8]:
import nltk.wsd


sent = sents[5]
words, tagged_words = tokenize_and_tag(sent)
wsd_sent =make_pywsd_sent(get_tagged_phrases(tagged_words,3))
print("\n".join(map(str,wsd_sent)))

(u'Is', u'Is', None)
(u'changing', u'changing', u'v')
(u'an', u'an', None)
(u'odd', u'odd', u'a')
(u'number', u'number', u'n')
(u'of', u'of', None)
(u'verbs', u'verbs', u'n')
(u',', u',', None)
(u'adverbs', u'adverbs', u'n')
(u',', u',', None)
(u'adjectives', u'adjectives', u'n')
(u'and', u'and', None)
(u'nouns', u'nouns', u'n')
(u'to', u'to', None)
(u'their', u'their', None)
(u'antonyms', u'antonyms', u'n')
(u'expected', u'expected', u'v')
(u'to', u'to', None)
(u'produce', u'produce', u'v')
(u'a', u'a', None)
(u'semantically', u'semantically', u'r')
(u'distant', u'distant', u'a')
(u'sentence', u'sentence', u'n')
(u'?', u'?', None)


In [11]:
tagged_words

[(u'Is', u'VBZ'),
 (u'changing', u'VBG'),
 (u'an', u'DT'),
 (u'odd', u'JJ'),
 (u'number', u'NN'),
 (u'of', u'IN'),
 (u'verbs', u'NNS'),
 (u',', u','),
 (u'adverbs', u'NNS'),
 (u',', u','),
 (u'adjectives', u'NNS'),
 (u'and', u'CC'),
 (u'nouns', u'NNS'),
 (u'to', u'TO'),
 (u'their', u'PRP$'),
 (u'antonyms', u'NNS'),
 (u'expected', u'VBN'),
 (u'to', u'TO'),
 (u'produce', u'VB'),
 (u'a', u'DT'),
 (u'semantically', u'RB'),
 (u'distant', u'JJ'),
 (u'sentence', u'NN'),
 (u'?', u'.')]

In [10]:
ans=disambiguate(wsd_sent, context_is_lemmatized=True, nbest=True, normalizescore=True, keepscore=True, algorithm=pywsd.lesk.simple_lesk)
ans

[(u'Is', None),
 (u'changing',
  [(0.2, Synset('switch.v.03')),
   (0.2, Synset('exchange.v.01')),
   (0.2, Synset('change.v.06')),
   (0.2, Synset('change.v.03')),
   (0.2, Synset('change.v.02')),
   (0.0, Synset('transfer.v.06')),
   (0.0, Synset('deepen.v.04')),
   (0.0, Synset('change.v.10')),
   (0.0, Synset('change.v.05')),
   (0.0, Synset('change.v.01'))]),
 (u'an', None),
 (u'odd', [(1.0, Synset('odd.a.01'))]),
 (u'number',
  [(0.09090909090909091, Synset('phone_number.n.01')),
   (0.09090909090909091, Synset('numeral.n.01')),
   (0.09090909090909091, Synset('number.n.11')),
   (0.09090909090909091, Synset('number.n.10')),
   (0.09090909090909091, Synset('number.n.09')),
   (0.09090909090909091, Synset('number.n.08')),
   (0.09090909090909091, Synset('number.n.07')),
   (0.09090909090909091, Synset('number.n.02')),
   (0.09090909090909091, Synset('number.n.01')),
   (0.09090909090909091, Synset('issue.n.02')),
   (0.09090909090909091, Synset('act.n.04'))]),
 (u'of', None),
 (u'

In [51]:
ss=ans[-2][1][2][1]
ss.wup_similarity

<bound method Synset.wup_similarity of Synset('conviction.n.02')>

In [30]:
ans[-2][1].definition()

u'a string of words satisfying the grammatical rules of a language'

In [96]:
s.

[]

In [35]:
wndomains.Syns

[]

In [10]:
for sent in sents:
    tagged_phrases = get_tagged_phrases(tokenize_and_tag(sent)[1],3)
    phrases = zip(*tagged_phrases)[0]
    print " ".join(phrases)

It 's not too_bad but makes a_lot of unnatural sounding sentences .
A Washington County man may have the countys first human case of West_Nile_virus , the health department said Friday
It was a dusty , dry , hot day , and the flys were buzzing .
The article is the most common determiner ( DT ) in English .
We may have a question
Is changing an odd number of verbs , adverbs , adjectives and nouns to their antonyms expected to produce a semantically distant sentence ?
Both gerunds and infinitives can be used as the subject or the complement of a sentence .
Shares of Xoma fell 16 percent in early trade , while shares of Genentech , a much larger company with several products on the market , were up 2 percent .
Six months ago , the IMF and Argentina struck a bare-minimum $ 6.8-billion debt rollover deal that expires in August .
He plans to have dinner with troops at Kosovo 's U.S. military_headquarters , Camp Bondsteel .
After that , he plans to have dinner at Camp Bondsteel with U.S. troo

In [24]:
for sent in sents:
    indexes = list(get_phrases_indexes(tokenize_and_tag(sent)[1],3))
    indexes.sort()
    print indexes

[3, 4, 7, 8]
[12, 13, 14]
[]
[]
[]
[]
[]
[]
[]
[11, 12]
[]
[7, 8]
[]
[]
[]
[]
[]
[]
[9, 10]
[2, 3, 4, 15, 16, 17, 24, 25]


In [15]:
a= set()

In [20]:
a.update(["21"])

In [21]:
a

{'21'}

In [96]:
wn.synsets("face")

[Synset('face.n.01'),
 Synset('expression.n.01'),
 Synset('face.n.03'),
 Synset('face.n.04'),
 Synset('face.n.05'),
 Synset('side.n.04'),
 Synset('face.n.07'),
 Synset('face.n.08'),
 Synset('grimace.n.01'),
 Synset('font.n.01'),
 Synset('face.n.11'),
 Synset('boldness.n.02'),
 Synset('face.n.13'),
 Synset('confront.v.02'),
 Synset('confront.v.01'),
 Synset('front.v.01'),
 Synset('face.v.04'),
 Synset('face.v.05'),
 Synset('confront.v.03'),
 Synset('face.v.07'),
 Synset('face.v.08'),
 Synset('face.v.09')]

In [123]:
s=wn.synset('bank.n.01')
s.hypernyms()

[Synset('slope.n.01')]

In [369]:
# Loading the Wordnet domains.
from collections import defaultdict

from glob import glob
import os.path


domain_files = glob('../../../Resources/tools/lib/databases/xwnd-30g/*.ppv')
synset2domains = defaultdict(lambda: np.zeros(len(domain_files)))
for domain_index,domain_filename in enumerate(domain_files):
    domain = os.path.splitext(os.path.basename(domain_filename))[0]
    with open(domain_filename,'r') as fh:
        for line in list(fh):
            ssid, weight = line.split()
            weight = float(weight)
            #assert not np.isnan(weight)
            
            synset2domains[ssid][domain_index] = weight 


def get_domains(synset):
    ssid = str(synset.offset()).zfill(8) + "-" + synset.pos()
    return synset2domains[ssid]
    

In [334]:
s=wn.synsets("bank")[1]
s

Synset('depository_financial_institution.n.01')

In [337]:
doms = get_domain(s)
doms.shape

(170,)

In [293]:
doms.most_common(3)

[('banking', 0.0150926), ('finance', 0.00172323), ('book_keeping', 0.00112378)]

In [237]:
s.definition()

u'a string of words satisfying the grammatical rules of a language'

In [304]:
c=Counter()
c["a"]=0.22
c["b"]=0.23
c["b"]+=0.13
d=Counter()
d["a"]=-0.21
d["b"]=1.2


In [312]:
e=(c+d)
e+=c


TypeError: unsupported operand type(s) for /: 'Counter' and 'int'

In [296]:
c.most_common(1)

[('b', 0.36)]

In [404]:
def domain_wsd(sent, max_phrase_len=3):
    def full_penn2wn_pos(penn):
        tag = penn[0].lower()
        tag = 'a' if tag =='j' else tag
        return tag if tag in ['a','v','r','n'] else None
        
    words, tagged_words = tokenize_and_tag(sent)
    tagged_phrases = get_tagged_phrases(tagged_words,max_phrase_len)
    
    sentence_domain = np.zeros(len(domain_files))
    
    for lemma,pos in tagged_phrases:
        wn_pos = full_penn2wn_pos(pos)
        if wn_pos: 
            for ss in wn.synsets(lemma, wn_pos):
                sentence_domain+=get_domains(ss)
    
    def get_best_synsets():
        for lemma,pos in tagged_phrases:
            best_synset = None
            wn_pos = full_penn2wn_pos(pos)
            if wn_pos:
                best_score = -1*np.Inf
                for ss in wn.synsets(lemma, pennPOS2WordnetPOS(wn_pos)):
                    score = np.dot(get_domains(ss),sentence_domain)
                    if score>best_score:
                        best_score=score
                        best_synset=ss
            yield lemma, best_synset, None if best_synset is None else best_synset.definition()
            
    return list(get_best_synsets())

In [409]:
sent = sents[19]
print sent
print "--------------"
sent_dom = domain_wsd(sent)
sent_dom

PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer.
--------------


[(u'PCCW', None, None),
 (u"'s", None, None),
 (u'chief_operating_officer',
  Synset('chief_executive_officer.n.01'),
  u'the corporate executive responsible for the operations of the firm; reports to a board of directors; may appoint other managers (including a president)'),
 (u',', None, None),
 (u'Mike',
  Synset('microphone.n.01'),
  u'device for converting sound waves into electrical energy'),
 (u'Butcher',
  Synset('butcher.v.01'),
  u'kill (animals) usually for food consumption'),
 (u',', None, None),
 (u'and', None, None),
 (u'the', None, None),
 (u'Arena',
  Synset('sphere.n.01'),
  u'a particular environment or walk of life'),
 (u'brothers',
  Synset('brother.n.05'),
  u'(Roman Catholic Church) a title given to a monk and used as form of address'),
 (u',', None, None),
 (u'the', None, None),
 (u'chief_financial_officers',
  Synset('chief_financial_officer.n.01'),
  u'the corporate executive having financial authority to make appropriations and authorize expenditures for a fir

In [411]:
!git commit -m="I may have (but probably have not) created a new method for doing word sense disambiguation"

[master d2da037] =I may have (but probably have not) created a new method for doing word sense disambiguation
 1 file changed, 496 insertions(+), 7 deletions(-)
